# 4. Batch Inference Pipeline

## 4.1. Setup

### 4.1.1. Import Libraries

In [37]:
# Standard imports
import os
from pathlib import Path
import sys
import json
import time
from datetime import date, datetime, timedelta
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore", module="IPython")
warnings.filterwarnings("ignore", category=DeprecationWarning)

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import numpy as np
import great_expectations as gx
import hopsworks
from urllib3.exceptions import ProtocolError
from requests.exceptions import ConnectionError, Timeout, RequestException
from confluent_kafka import KafkaException
from hsfs.client.exceptions import RestAPIError
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from scipy.spatial.distance import cdist

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata, visualization

today = datetime.today().date()

Project root dir: c:\Users\krist\Documents\GitHub\pm25


### 4.1.2. Initialize Hopsworks Connection

In [38]:

def detect_environment():
    if (
        "HOPSWORKS_JOB_ID" in os.environ
        or "HOPSWORKS_PROJECT_ID" in os.environ
        or "HOPSWORKS_JOB_NAME" in os.environ
    ):
        return "job"

    cwd = os.getcwd()
    if cwd.startswith("/hopsfs/Jupyter"):
        return "jupyter"

    return "local"

env = detect_environment()
print(f"Detected environment: {env}")

# Load secrets based on environment

if env in ("job", "jupyter"):
    project = hopsworks.login()
    secrets_api = hopsworks.get_secrets_api()

    for key in ["HOPSWORKS_API_KEY", "AQICN_API_KEY", "GH_PAT", "GH_USERNAME"]:
        os.environ[key] = secrets_api.get_secret(key).value

else:
    load_dotenv()

# Load Pydantic settings

settings = config.HopsworksSettings()

HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks using the API key

project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

print("Environment initialized and Hopsworks connected!")


Detected environment: local
HopsworksSettings initialized!
2026-01-29 11:17:56,652 INFO: Closing external client and cleaning up certificates.
Connection closed.
2026-01-29 11:17:56,668 INFO: Initializing external client
2026-01-29 11:17:56,670 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-29 11:17:58,240 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184
Environment initialized and Hopsworks connected!


### 4.1.3. Repository management

In [39]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

💻 Already in git repository at c:\Users\krist\Documents\GitHub\pm25


### 4.1.4. Configure API Keys and Secrets

In [40]:
secrets = hopsworks.get_secrets_api()

try:
    secrets.get_secret("AQICN_API_KEY")
except:
    secrets.create_secret("AQICN_API_KEY", settings.AQICN_API_KEY.get_secret_value())

### 4.1.5. Get Model Registry

In [41]:
mr = project.get_model_registry()

## 4.2. Get Feature Groups and Sensor Locations

In [ ]:
air_quality_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

# # Reload metadata module to pick up new functions
# import importlib
# importlib.reload(metadata)

# Load data from air_quality feature group
aq_data = air_quality_fg.read()

if len(aq_data) == 0:
    print("⚠️ No air quality data found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

sensor_locations = metadata.get_sensor_locations(air_quality_fg)
print(f"📍 Loaded locations for {len(sensor_locations)} sensors")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.08s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.44s) 
📍 Loaded locations for 103 sensors


In [45]:

# # Build sensor location dictionary: sensor_id -> (lat, lon, city, street, country, aqicn_url)
# sensor_locations = {}
# existing_aq_data = air_quality_fg.read()
# existing_sensors = set(existing_aq_data["sensor_id"].unique())
# print(f"📋 Found {len(existing_sensors)} sensors in feature store")

# # Build location dict
# for _, row in existing_aq_data[["sensor_id", "latitude", "longitude", "city", "street", "country"]].drop_duplicates(subset=["sensor_id"]).iterrows():
#     sensor_locations[row["sensor_id"]] = (
#         row["latitude"], 
#         row["longitude"], 
#         row["city"], 
#         row["street"], 
#         row["country"]
#     )
# print(f"📍 Loaded locations for {len(sensor_locations)} existing sensors")

### 4.2.3. Load Sensor Locations

In [46]:
# all_secrets = secrets.get_secrets()
# locations = {}
# for secret in all_secrets:
#     if secret.name.startswith("SENSOR_LOCATION_JSON_"):
#         sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
#         location_str = secrets.get_secret(secret.name).value
#         if location_str:
#             locations[sensor_id] = json.loads(location_str)
# print(f"Retrieved locations for {len(locations)} sensors from Hopsworks Secrets Manager.")


# # Convert locations to use integer keys to match sensor_locations
# locations_int = {}
# for sensor_id_str, location_data in locations.items():
#     try:
#         sensor_id_int = int(sensor_id_str)
#         locations_int[sensor_id_int] = location_data
#     except ValueError:
#         print(f"⚠️ Could not convert sensor ID '{sensor_id_str}' to integer")

# # Use integer-keyed version throughout
# locations = locations_int
# print(f"Converted to {len(locations)} integer-keyed location entries")

## 4.3. Load Data from Feature Store

### 4.3.1. Set Inference Dates

In [47]:
past_date = today - timedelta(days=7)  # Get 7 days of historical data for feature engineering
future_date = today + timedelta(days=7)  # Get 7 days of future weather forecasts
today_short = today.strftime("%Y-%m-%d")

print(f"Inference period: {past_date} to {future_date}")
print(f"Today: {today_short}")

Inference period: 2026-01-22 to 2026-02-05
Today: 2026-01-29


### 4.3.2. Load Weather Features

In [48]:
try:
    batch_weather = weather_fg.filter(
        (weather_fg.date >= past_date) & (weather_fg.date <= future_date)
    ).read()
except Exception:
    batch_weather = weather_fg.read()
    batch_weather = batch_weather[
        (batch_weather["date"] >= past_date) & (batch_weather["date"] <= future_date)
    ]

batch_weather["date"] = pd.to_datetime(batch_weather["date"]).dt.tz_localize(None)

print(f"Retrieved {len(batch_weather)} weather records from {past_date} to {future_date}")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 
Retrieved 1339 weather records from 2026-01-22 to 2026-02-05


### 4.3.3. Load Air Quality Features

In [49]:
try:
    batch_airquality = air_quality_fg.filter(
        air_quality_fg.date >= past_date
    ).read()
except Exception:
    # batch_airquality = pd.DataFrame()
    batch_airquality = air_quality_fg.read()
    batch_airquality = batch_airquality[
        batch_airquality["date"] >= past_date
    ]

batch_airquality["date"] = pd.to_datetime(batch_airquality["date"]).dt.tz_localize(None)

print(f"Retrieved {len(batch_airquality)} air quality records from {past_date} to {today}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.06s) 
Retrieved 0 air quality records from 2026-01-22 to 2026-01-29


## 4.4. Model Retrieval
Download trained XGBoost models from Hopsworks model registry for each sensor and extract feature names.

### 4.4.1. Load Model Registry Metadata

In [50]:
MODEL_NAME_TEMPLATE = "air_quality_xgboost_model_{sensor_id}"

retrieved_models = {}


# Use sensor_locations from feature store instead of locations from secrets
for sensor_id in sensor_locations.keys():  # ✅ Use feature store sensors
    model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
    
    try:
        available_models = mr.get_models(name=model_name)
        if not available_models:
            print(f"⚠️ No model found for sensor {sensor_id}, skipping...")
            continue
        
        retrieved_model = max(available_models, key=lambda model: model.version)
        saved_model_dir = retrieved_model.download()
        
        xgb_model = XGBRegressor()
        xgb_model.load_model(saved_model_dir + "/model.json")
        booster = xgb_model.get_booster()
        
        retrieved_models[sensor_id] = retrieved_model, xgb_model, booster.feature_names
        
    except Exception as e:
        print(f"❌ Error loading model for sensor {sensor_id}: {e}")
        continue

print(f"✅ Retrieved {len(retrieved_models)} models from registry")
print(f"   Total sensors in feature store: {len(sensor_locations)}")

# for sensor_id in locations.keys():
#     model_name = MODEL_NAME_TEMPLATE.format(sensor_id=sensor_id)
#     retrieved_model = None

#     available_models = mr.get_models(name=model_name)
#     if available_models:
#         retrieved_model = max(available_models, key=lambda model: model.version)

#     if retrieved_model is None:
#         print(f"No model found for sensor {sensor_id}, skipping...")
#         continue
    
#     saved_model_dir = retrieved_model.download()  
    
#     xgb_model = XGBRegressor()
#     xgb_model.load_model(saved_model_dir + "/model.json")
#     booster = xgb_model.get_booster()
    
#     # booster = xgb.Booster()
#     # booster.load_model(saved_model_dir + "/model.json")
#     # xgb_model = XGBRegressor()
#     # xgb_model._Booster = booster

#     retrieved_models[sensor_id] = retrieved_model, xgb_model, booster.feature_names

Downloading: 0.000%|          | 0/515507 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/135604 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/41497 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/40866 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/18687 elapsed<00:00 remaining<?

...

Downloading: 0.000%|          | 0/85905 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/41449 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/37172 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/23841 elapsed<00:00 remaining<?

✅ Retrieved 103 models from registry5 files)... DONE
   Total sensors in feature store: 103


In [51]:
print(f"Retrieved {len(retrieved_models)} models.")

Retrieved 103 models.


### 4.4.2. Retrieve Best Model per Sensor

In [52]:

# Already done in 4.4.1 - models retrieved by max version
# retrieved_models dict contains: {sensor_id: (model, xgb_model, feature_names)}


### 4.4.3. Load Model Artifacts
XGBoost, preprocessing, etc.

In [53]:

# Already loaded in 4.4.1 using:
# xgb_model.load_model(saved_model_dir + "/model.json")

## 4.5. Batch Prediction
Merge weather and air quality data, iteratively predict PM2.5 values for forecast days, update engineered features after each prediction, and store results

### 4.5.1. Batch Prediction Loop

In [54]:
PREDICTION_CAP_MAX = 150.0  # Maximum reasonable PM2.5 value
PREDICTION_CAP_MIN = 0.0    # Minimum reasonable PM2.5 value

In [55]:
# Merge historical data with weather data
batch_data = pd.merge(batch_weather, batch_airquality, on=["date", "sensor_id"], how="left")
batch_data = batch_data.sort_values(["sensor_id", "date"])

In [56]:
feature_cols = [
    "pm25_rolling_3d",
    "pm25_lag_1d",
    "pm25_lag_2d",
    "pm25_lag_3d",
    "pm25_nearby_avg",
]

In [57]:
# Create a new column for pedicted pm2.5 values, fill with NaN for now
batch_data["predicted_pm25"] = np.nan
# Create a new column for days before forecast day, fill with NaN for now
batch_data["days_before_forecast_day"] = np.nan
# For each feature name in feature_cols list, 
# create a new column for predicted feature values, fill with NaN for now
for col in feature_cols:
    batch_data[f"predicted_{col}"] = np.nan
# Select all rows where pm25 is NaN and date is today or later
# drop any NaN date values, sort the dates in ascending order, get unique dates
# forecast days will be a list of dates for which pm2.5 predictions are needed
forecast_days = (
    batch_data.loc[batch_data["pm25"].isna() & 
                   (batch_data["date"] >= today.strftime("%Y-%m-%d")), "date"]
    .dropna()
    .sort_values()
    .unique()
)

In [58]:
# Track sensors processed to prevent duplicates and count progress
# sensors_processed = set()
# warning_count = 0
# MAX_WARNINGS = 3  # Reduced to minimize output noise
# predictions_made = 0

In [59]:
# print(forecast_days)

In [60]:
for target_day in forecast_days:
    # context with all sensors up to current day
    window = batch_data.loc[batch_data["date"] <= target_day].copy()
    day_rows = window[(window["date"] == target_day) & window["pm25"].isna()]

    for _, row in day_rows.iterrows():
        sensor_id = row["sensor_id"]
        try:
            _, xgb_model, model_features = retrieved_models[sensor_id]
        except KeyError:
            print(f"No model for sensor {sensor_id}, skipping prediction for {target_day}.")
            continue
        features = (row.reindex(model_features).to_frame().T.apply(pd.to_numeric, errors="coerce"))
        y_hat = xgb_model.predict(features)[0]

        idx = batch_data.index[(batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)][0]
        batch_data.at[idx, "pm25"] = y_hat
        batch_data.at[idx, "predicted_pm25"] = y_hat
        batch_data.at[idx, "days_before_forecast_day"] = (target_day - pd.Timestamp(today)).days + 1

    # Recompute features for after filling this day
    temp_df = batch_data.loc[batch_data["date"] <= target_day].copy()
    temp_df = feature_engineering.add_rolling_window_feature(
        temp_df, window_days=3, column="pm25", new_column="pm25_rolling_3d"
    )
    temp_df = feature_engineering.add_lagged_features(temp_df, column="pm25", lags=[1, 2, 3])
    temp_df = feature_engineering.add_nearby_sensor_feature(
        temp_df,
        locations,
        column="pm25_lag_1d",
        n_closest=3,
        new_column="pm25_nearby_avg",
    )

    current_rows = temp_df[temp_df["date"] == target_day]
    for _, row in current_rows.iterrows():
        sensor_id = row["sensor_id"]
        mask = (batch_data["sensor_id"] == sensor_id) & (batch_data["date"] == target_day)
        if mask.any():
            for col in feature_cols:
                batch_data.loc[mask, f"predicted_{col}"] = row[col]

predictions = batch_data.loc[
    batch_data["predicted_pm25"].notna(),
    ["date", "sensor_id", "predicted_pm25", "days_before_forecast_day"]
    + [f"predicted_{col}" for col in feature_cols],
].reset_index(drop=True)
batch_data.loc[batch_data["date"] > pd.Timestamp(today), "pm25"] = np.nan

In [61]:
# # Add after model retrieval section
# all_models = mr.get_models()
# print(f"Total models in registry: {len(all_models)}")
# for model in all_models[:5]:  # Show first 5
#     print(f"  - {model.name} (v{model.version})")

### 4.5.2. Assemble Prediction Results

In [62]:
predictions_df = predictions.copy()

print(f"✅ Generated {len(predictions_df)} prediction rows")
print(f"   Date range: {predictions_df['date'].min()} to {predictions_df['date'].max()}")
print(f"   Sensors: {predictions_df['sensor_id'].nunique()}")
print(f"   Forecast days: {sorted(predictions_df['days_before_forecast_day'].unique())}")

✅ Generated 721 prediction rows
   Date range: 2026-01-29 00:00:00 to 2026-02-04 00:00:00
   Sensors: 103
   Forecast days: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]


### 4.5.3. Add Timestamps, Sensor IDs, and Metadata

*Should produce a clean predictions_df*

In [63]:
print(predictions_df.head())

        date  sensor_id  predicted_pm25  days_before_forecast_day  \
0 2026-01-29      57421        1.906243                       1.0   
1 2026-01-30      57421        4.558151                       2.0   
2 2026-01-31      57421        1.923128                       3.0   
3 2026-02-01      57421        1.743351                       4.0   
4 2026-02-02      57421        1.943291                       5.0   

   predicted_pm25_rolling_3d  predicted_pm25_lag_1d  predicted_pm25_lag_2d  \
0                        NaN                    NaN                    NaN   
1                   1.906243               1.906243                    NaN   
2                   3.232197               4.558151               1.906243   
3                   2.795840               1.923128               4.558151   
4                   2.741543               1.743351               1.923128   

   predicted_pm25_lag_3d  predicted_pm25_nearby_avg  
0                    NaN                        NaN  
1       

## 4.6. Save Predictions

### 4.6.1. Save Predictions to Feature Store / Model Registry / Dataset

In [64]:


# print("🔍 Diagnostic Info:")
# print(f"\nPredictions DataFrame columns: {predictions.columns.tolist()}")
# print(f"\nPredictions DataFrame shape: {predictions.shape}")
# print(f"\nSample data:")
# print(predictions.head())

# # Check if the feature group already exists and what schema it has
# try:
#     existing_fg = fs.get_feature_group("air_quality_predictions", version=1)
#     print(f"\n📋 Existing feature group schema:")
#     for feat in existing_fg.features:
#         print(f"  - {feat.name} ({feat.type})")
# except:
#     print("\n✅ Feature group doesn't exist yet - will be created fresh")

In [65]:
# # Delete existing feature group if it exists (development only!)
# try:
#     existing_fg = fs.get_feature_group("air_quality_predictions", version=1)
#     existing_fg.delete()
#     print("🗑️ Deleted old feature group")
# except:
#     pass

# Create new feature group with ONLY prediction columns
predictions_fg = fs.get_or_create_feature_group(
    name="air_quality_predictions",
    version=1,
    primary_key=["sensor_id", "date"],
    description="Daily PM2.5 predictions per sensor",
    event_time="date"
)

# Just use predictions as-is (keep the predicted_ prefix)
print(f"📊 Inserting {len(predictions)} prediction rows")
print(f"   Columns: {list(predictions.columns)}")
print(f"   Date range: {predictions['date'].min()} to {predictions['date'].max()}")

if len(predictions) > 0:
    predictions_fg.insert(predictions, write_options={"wait_for_job": False})
    print("✅ Insert successful!")
else:
    print("⚠️ No predictions to insert")

📊 Inserting 721 prediction rows
   Columns: ['date', 'sensor_id', 'predicted_pm25', 'days_before_forecast_day', 'predicted_pm25_rolling_3d', 'predicted_pm25_lag_1d', 'predicted_pm25_lag_2d', 'predicted_pm25_lag_3d', 'predicted_pm25_nearby_avg']
   Date range: 2026-01-29 00:00:00 to 2026-02-04 00:00:00


Uploading Dataframe: 100.00% |██████████| Rows 721/721 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_predictions_1_offline_fg_materialization/executions
✅ Insert successful!


## 4.7. Monitoring & Logging

### 4.7.1. Insert Monitoring Data

In [66]:
monitor_fg = fs.get_or_create_feature_group(
    name="aq_predictions",
    description="Air Quality prediction monitoring",
    version=1,
    primary_key=["sensor_id", "date", "days_before_forecast_day"],
    event_time="date",
)

monitor_fg.insert(predictions, wait=True)

Uploading Dataframe: 100.00% |██████████| Rows 721/721 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: aq_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/aq_predictions_1_offline_fg_materialization/executions
2026-01-29 11:38:50,600 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2026-01-29 11:38:53,826 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-29 11:40:33,951 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2026-01-29 11:40:34,359 INFO: Waiting for log aggregation to finish.
2026-01-29 11:40:34,360 INFO: Execution finished successfully.


(Job('aq_predictions_1_offline_fg_materialization', 'SPARK'), None)

### 4.7.2. Compute Monitoring Metrics (opt)

## 4.8. Analysis & Visualization

### 4.8.1. Generate Forecast Plots

In [67]:
forecast_paths = []

for sensor_id, location in locations.items():
    sensor_forecast = predictions[predictions["sensor_id"] == sensor_id].copy()

    city, street = location["city"], location["street"]
    forecast_path = f"{root_dir}/models/{sensor_id}/images/forecast.png"
    Path(forecast_path).parent.mkdir(parents=True, exist_ok=True)

    fig = visualization.plot_air_quality_forecast(
        location["city"],
        location["street"],
        sensor_forecast,
        forecast_path,
        hindcast=False,
    )
    plt.close(fig)
    forecast_paths.append((sensor_id, forecast_path))

if not dataset_api.exists("Resources/airquality"):
    dataset_api.mkdir("Resources/airquality")

for sensor_id, forecast_path in forecast_paths:
    dataset_api.upload(
        forecast_path,
        f"Resources/airquality/{sensor_id}_{today_short}_forecast.png",
        overwrite=True,
    )
print(f"Forecast plots available in Hopsworks under {project.get_url()}/settings/fb/path/Resources/airquality")

Uploading c:\Users\krist\Documents\GitHub\pm25/models/121810/images/forecast.png: 0.000%|          | 0/40733 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/192520/images/forecast.png: 0.000%|          | 0/38671 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/196735/images/forecast.png: 0.000%|          | 0/42693 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/208483/images/forecast.png: 0.000%|          | 0/38307 e…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/415030/images/forecast.png: 0.000%|          | 0/40457 e…

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/77488/images/forecast.png: 0.000%|          | 0/38845 el…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/79999/images/forecast.png: 0.000%|          | 0/40835 el…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/81505/images/forecast.png: 0.000%|          | 0/40177 el…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/92683/images/forecast.png: 0.000%|          | 0/35515 el…

Forecast plots available in Hopsworks under https://c.app.hopsworks.ai:443/p/1279184/settings/fb/path/Resources/airquality


### 4.8.2. Hindcast Analysis

In [68]:
try:
    check_df = fs.get_feature_group("aq_predictions", version=1).read()
    print(f"✅ aq_predictions feature group contains {len(check_df)} rows")
    
    # Check the data types and sample data
    if len(check_df) > 0:
        print(f"\nColumn types:")
        print(check_df.dtypes)
        print(f"\nSample rows:")
        print(check_df.head())
        print(f"\nUnique days_before_forecast_day values: {check_df['days_before_forecast_day'].unique()}")
    
except Exception as e:
    print(f"❌ Error reading aq_predictions feature group: {e}")
    check_df = pd.DataFrame()
        
if len(check_df) > 0:
    monitor_fg = fs.get_feature_group("aq_predictions", version=1)
    
    try:
        # Try reading without filter first
        print("Attempting to read monitoring data with filter...")
        monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
        print(f"✅ Successfully read {len(monitoring_df)} rows with filter")
        monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)
        
    except Exception as e:
        print(f"⚠️ Filter operation failed: {e}")
        print("Attempting to read all data and filter in pandas...")
        
        try:
            # Fallback: read all data and filter locally
            all_monitoring_df = monitor_fg.read()
            monitoring_df = all_monitoring_df[all_monitoring_df["days_before_forecast_day"] == 1].copy()
            monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)
            print(f"✅ Successfully filtered {len(monitoring_df)} rows locally")
            
        except Exception as e2:
            print(f"❌ Could not read monitoring data: {e2}")
            print("Skipping hindcast analysis...")
            monitoring_df = pd.DataFrame()

    if not monitoring_df.empty:
        air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
        air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

        hindcast_count = 0
        for sensor_id, location in locations.items():
            try:
                sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
                
                if sensor_preds.empty:
                    continue
                    
                merged = sensor_preds.merge(
                    air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
                    on="date",
                    how="inner",
                ).sort_values("date")

                city, street = location["city"], location["street"]
                hindcast_path = f"{root_dir}/models/{sensor_id}/images/hindcast_prediction.png"
                Path(hindcast_path).parent.mkdir(parents=True, exist_ok=True)

                fig = visualization.plot_air_quality_forecast(
                    city,
                    street,
                    merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
                    hindcast_path,
                    hindcast=True,
                )
                if fig is not None:
                    plt.close(fig)

                dataset_api.upload(
                    hindcast_path,
                    f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
                    overwrite=True,
                )
                
                hindcast_count += 1

            except Exception as e:
                print(f"⚠️ Error processing hindcast for sensor {sensor_id}: {e}")
        
        print(f"✅ Generated {hindcast_count} hindcast plots")
    else:
        print("⚠️ No monitoring data available for hindcast analysis")
else:
    print("⚠️ Skipping hindcast analysis - no data in aq_predictions feature group")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.20s) 
✅ aq_predictions feature group contains 44925 rows

Column types:
date                         datetime64[us, Etc/UTC]
sensor_id                                      int32
predicted_pm25                               float64
days_before_forecast_day                     float64
predicted_pm25_rolling_3d                    float64
predicted_pm25_lag_1d                        float64
predicted_pm25_lag_2d                        float64
predicted_pm25_lag_3d                        float64
predicted_pm25_nearby_avg                    float64
dtype: object

Sample rows:
                       date  sensor_id  predicted_pm25  \
0 2025-09-30 00:00:00+00:00     497266        1.329760   
1 2025-05-01 00:00:00+00:00      65290        0.010900   
2 2024-12-09 00:00:00+00:00      61420        1.217872   
3 2024-08-05 00:00:00+00:00     194215        5.002939   
4 2025-11-16 00:00:00+00:00     180187        0.12876

Uploading c:\Users\krist\Documents\GitHub\pm25/models/121810/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/192520/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/196735/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/208483/images/hindcast_prediction.png: 0.000%|          …

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/77488/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/79999/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/81505/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/92683/images/hindcast_prediction.png: 0.000%|          |…

✅ Generated 102 hindcast plots


In [69]:
monitor_fg = fs.get_feature_group("aq_predictions", version=1)  # Changed from "monitor"

try:
    monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast_day == 1).read()
    monitoring_df["date"] = pd.to_datetime(monitoring_df["date"]).dt.tz_localize(None)
except Exception as e:
    print(f"⚠️ Could not read monitoring data: {e}")
    print("Skipping hindcast analysis...")
    monitoring_df = pd.DataFrame()  # Empty dataframe to prevent further errors

if not monitoring_df.empty:
    air_quality_df = air_quality_fg.read()[["date", "sensor_id", "pm25"]]
    air_quality_df["date"] = pd.to_datetime(air_quality_df["date"]).dt.tz_localize(None)

    for sensor_id, location in locations.items():
        try:
            sensor_preds = monitoring_df[monitoring_df["sensor_id"] == sensor_id][["date", "predicted_pm25"]]
            
            if sensor_preds.empty:
                continue
                
            merged = sensor_preds.merge(
                air_quality_df[air_quality_df["sensor_id"] == sensor_id][["date", "pm25"]],
                on="date",
                how="inner",
            ).sort_values("date")

            city, street = location["city"], location["street"]
            hindcast_path = f"{root_dir}/models/{sensor_id}/images/hindcast_prediction.png"
            Path(hindcast_path).parent.mkdir(parents=True, exist_ok=True)

            fig = visualization.plot_air_quality_forecast(
                city,
                street,
                merged if not merged.empty else sensor_preds.assign(pm25=np.nan),
                hindcast_path,
                hindcast=True,
            )
            if fig is not None:
                plt.close(fig)

            dataset_api.upload(
                hindcast_path,
                f"Resources/airquality/{sensor_id}_{today:%Y-%m-%d}_hindcast.png",
                overwrite=True,
            )

        except Exception as e:
            print(f"⚠️ Error processing hindcast for sensor {sensor_id}: {e}")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.20s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.66s) 


Uploading c:\Users\krist\Documents\GitHub\pm25/models/121810/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/192520/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/196735/images/hindcast_prediction.png: 0.000%|          …

Uploading c:\Users\krist\Documents\GitHub\pm25/models/208483/images/hindcast_prediction.png: 0.000%|          …

...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/70564/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/77488/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/79999/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/81505/images/hindcast_prediction.png: 0.000%|          |…

Uploading c:\Users\krist\Documents\GitHub\pm25/models/92683/images/hindcast_prediction.png: 0.000%|          |…

### 4.8.3. IDW Heatmap

#### 4.8.3.1. IDW Interpolation Function

In [70]:
def idw_interpolation(points, values, grid_points, lon_mesh, power=2):
    # compute distances between grid points and known data points 
    distances = cdist(grid_points, points)
    # replace 0 with a small value to avoid division by zero
    distances = np.where(distances == 0, 1e-10, distances)
    # compute weights based on inverse distance
    weights = 1.0 / (distances ** power)
    # sum of weights for normalization
    weights_sum = np.sum(weights, axis=1)
    # compute interpolated values - weighted average of known values for each grid point
    interpolated = np.sum(weights * values, axis=1) / weights_sum
    # reshape to the match grid shape
    return interpolated.reshape(lon_mesh.shape)

#### 4.8.3.2. Generate Heatmap Images 

In [71]:
grid_bounds = tuple(list(json.load(open(f"{root_dir}/frontend/coordinates.json")).values())[:4])
# grid_bounds = map_bounds[1], map_bounds[0], map_bounds[3], map_bounds[2]  # lat_min, lat_max, lon_min, lon_max
print(grid_bounds)

(-7.602536, 50.862218, 36.738284, 69.923179)


In [72]:
interpolation_dir = f"{root_dir}/models/interpolation"
os.makedirs(interpolation_dir, exist_ok=True)

interpolation_df = predictions.copy()

In [73]:
def plot_pm25_idw_heatmap(
    predictions: pd.DataFrame,
    locations: dict,
    forecast_date: datetime,
    path: str,
    grid_bounds=(11.4, 57.15, 12.5, 58.25),
    grid_resolution=800,
    power=2,
):

    df_day = predictions[predictions["date"] == forecast_date].copy()

    # Build sensor coordinates and PM2.5 values
    sensor_coords_list = []
    pm25_values_list = []
    
    pm25_column = "predicted_pm25" if not df_day["predicted_pm25"].isna().all() else "pm25"
    
    for sid in df_day["sensor_id"].unique():
        if sid in locations:
            sensor_coords_list.append([locations[sid]["longitude"], locations[sid]["latitude"]])
            pm25_val = df_day[df_day["sensor_id"] == sid][pm25_column].iloc[0]
            pm25_values_list.append(pm25_val)
    
    # Convert to numpy arrays
    sensor_coords = np.array(sensor_coords_list)
    pm25_values = np.array(pm25_values_list)
    
    # Safety check: need at least 1 sensor with data
    if len(sensor_coords) == 0 or len(pm25_values) == 0:
        print(f"⚠️ No sensor data available for {forecast_date}, skipping heatmap generation")
        return

    
    # Ensure sensor_coords is 2D (required by cdist)
    if sensor_coords.ndim == 1:
        sensor_coords = sensor_coords.reshape(1, -1)

    min_lon, min_lat, max_lon, max_lat = grid_bounds

    lon_grid = np.linspace(min_lon, max_lon, grid_resolution)
    lat_grid = np.linspace(min_lat, max_lat, grid_resolution)
    lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
    grid_points = np.column_stack([lon_mesh.ravel(), lat_mesh.ravel()])

    idw_result = idw_interpolation(sensor_coords, pm25_values, grid_points, lon_mesh, power=power)

    default_levels = np.array([0, 12, 35, 55, 150, 250, 500])
    category_colors = ["#00e400", "#7de400", "#ffff00", "#ffb000", "#ff7e00", "#ff4000", "#ff0000", "#c0007f", "#8f3f97", "#7e0023"]
    vmin, vmax = default_levels[0], 150
    
    clipped = np.clip(idw_result, vmin, vmax)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(
        clipped,
        extent=(min_lon, max_lon, min_lat, max_lat),
        origin="lower",
        cmap=mcolors.LinearSegmentedColormap.from_list("aqi", category_colors, N=512),
        vmin=vmin,
        vmax=vmax,
        alpha=0.5,
    )
    ax.set_xlim(min_lon, max_lon)
    ax.set_ylim(min_lat, max_lat)
    ax.axis("off")

    fig.savefig(path, dpi=300, bbox_inches="tight", pad_inches=0, transparent=True)
    plt.close(fig)

In [74]:

# Add any actual PM2.5 data from today if available
today_actual = batch_data[batch_data["date"] == today_short].copy()

if not today_actual.empty:
    # Ensure both columns exist for the plotting function
    today_actual = today_actual[[col for col in ["date", "sensor_id", "pm25", "predicted_pm25"] if col in today_actual.columns]]
    interpolation_df = pd.concat([today_actual, interpolation_df], ignore_index=True)

for i, forecast_date in enumerate(sorted(interpolation_df["date"].unique())):
    forecast_date_short = forecast_date.strftime("%Y-%m-%d")
    # output_png = f"{interpolation_dir}/forecast_interpolation_{i}d.png"
    days_ahead = (forecast_date - pd.Timestamp(today)).days
    output_png = f"{interpolation_dir}/forecast_interpolation_{days_ahead}d.png"
    print(interpolation_df.info())

    print("Exists before plotting:", os.path.exists(output_png))


    plot_pm25_idw_heatmap(
        interpolation_df,
        locations,
        forecast_date,
        output_png,
    )

    dataset_api.upload(
        output_png,
        f"Resources/airquality/interpolation_{today_short}_{forecast_date_short}.png",
        overwrite=True,
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       824 non-null    datetime64[us]
 1   sensor_id                  824 non-null    int32         
 2   pm25                       103 non-null    float64       
 3   predicted_pm25             824 non-null    float64       
 4   days_before_forecast_day   721 non-null    float64       
 5   predicted_pm25_rolling_3d  618 non-null    float64       
 6   predicted_pm25_lag_1d      618 non-null    float64       
 7   predicted_pm25_lag_2d      515 non-null    float64       
 8   predicted_pm25_lag_3d      412 non-null    float64       
 9   predicted_pm25_nearby_avg  612 non-null    float64       
dtypes: datetime64[us](1), float64(8), int32(1)
memory usage: 61.3 KB
None
Exists before plotting: True


Uploading c:\Users\krist\Documents\GitHub\pm25/models/interpolation/forecast_interpolation_0d.png: 0.000%|    …

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       824 non-null    datetime64[us]
 1   sensor_id                  824 non-null    int32         
 2   pm25                       103 non-null    float64       
 3   predicted_pm25             824 non-null    float64       
 4   days_before_forecast_day   721 non-null    float64       
 5   predicted_pm25_rolling_3d  618 non-null    float64       
 6   predicted_pm25_lag_1d      618 non-null    float64       
 7   predicted_pm25_lag_2d      515 non-null    float64       
 8   predicted_pm25_lag_3d      412 non-null    float64       
 9   predicted_pm25_nearby_avg  612 non-null    float64       
dtypes: datetime64[us](1), float64(8), int32(1)
memory usage: 61.3 KB
None
Exists before plotting: True


Uploading c:\Users\krist\Documents\GitHub\pm25/models/interpolation/forecast_interpolation_1d.png: 0.000%|    …

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       824 non-null    datetime64[us]
 1   sensor_id                  824 non-null    int32         
 2   pm25                       103 non-null    float64       
 3   predicted_pm25             824 non-null    float64       
 4   days_before_forecast_day   721 non-null    float64       
 5   predicted_pm25_rolling_3d  618 non-null    float64       
 6   predicted_pm25_lag_1d      618 non-null    float64       
 7   predicted_pm25_lag_2d      515 non-null    float64       
 8   predicted_pm25_lag_3d      412 non-null    float64       
 9   predicted_pm25_nearby_avg  612 non-null    float64       
dtypes: datetime64[us](1), float64(8), int32(1)
memory usage: 61.3 KB
None
Exists before plotting: True


...

Uploading c:\Users\krist\Documents\GitHub\pm25/models/interpolation/forecast_interpolation_4d.png: 0.000%|    …

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       824 non-null    datetime64[us]
 1   sensor_id                  824 non-null    int32         
 2   pm25                       103 non-null    float64       
 3   predicted_pm25             824 non-null    float64       
 4   days_before_forecast_day   721 non-null    float64       
 5   predicted_pm25_rolling_3d  618 non-null    float64       
 6   predicted_pm25_lag_1d      618 non-null    float64       
 7   predicted_pm25_lag_2d      515 non-null    float64       
 8   predicted_pm25_lag_3d      412 non-null    float64       
 9   predicted_pm25_nearby_avg  612 non-null    float64       
dtypes: datetime64[us](1), float64(8), int32(1)
memory usage: 61.3 KB
None
Exists before plotting: True


Uploading c:\Users\krist\Documents\GitHub\pm25/models/interpolation/forecast_interpolation_5d.png: 0.000%|    …

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       824 non-null    datetime64[us]
 1   sensor_id                  824 non-null    int32         
 2   pm25                       103 non-null    float64       
 3   predicted_pm25             824 non-null    float64       
 4   days_before_forecast_day   721 non-null    float64       
 5   predicted_pm25_rolling_3d  618 non-null    float64       
 6   predicted_pm25_lag_1d      618 non-null    float64       
 7   predicted_pm25_lag_2d      515 non-null    float64       
 8   predicted_pm25_lag_3d      412 non-null    float64       
 9   predicted_pm25_nearby_avg  612 non-null    float64       
dtypes: datetime64[us](1), float64(8), int32(1)
memory usage: 61.3 KB
None
Exists before plotting: True


Uploading c:\Users\krist\Documents\GitHub\pm25/models/interpolation/forecast_interpolation_6d.png: 0.000%|    …